In [1]:
from model.baseline import *
from model.load import *

In [9]:
DATASET = "extracted_audio"
epochs = 50

callbacks = [tf.keras.callbacks.ReduceLROnPlateau()]

In [3]:
x_train, y_train, x_test, y_test = loadSpectrogram(DATASET)

In [10]:
train, validation = loadDataset(x_train, y_train, x_test, y_test)

In [7]:
model = network()

In [ ]:
history = model.fit(train, validation_data = validation, epochs = 50, callbacks = callbacks,
                    class_weight = {0:1.,1:1.})

Train for 438 steps, validate for 110 steps
Epoch 1/50
438/438 [==============================] - 278s 635ms/step - loss: 0.4326 - accuracy: 0.8015 - val_loss: 0.4091 - val_accuracy: 0.8114
Epoch 2/50
438/438 [==============================] - 372s 849ms/step - loss: 0.3500 - accuracy: 0.8329 - val_loss: 0.3799 - val_accuracy: 0.8329
Epoch 3/50
438/438 [==============================] - 486s 1s/step - loss: 0.3289 - accuracy: 0.8458 - val_loss: 0.3480 - val_accuracy: 0.8414
Epoch 4/50
438/438 [==============================] - 533s 1s/step - loss: 0.3102 - accuracy: 0.8541 - val_loss: 0.3445 - val_accuracy: 0.8383
Epoch 5/50
438/438 [==============================] - 537s 1s/step - loss: 0.2910 - accuracy: 0.8638 - val_loss: 0.3388 - val_accuracy: 0.8474
Epoch 6/50
438/438 [==============================] - 523s 1s/step - loss: 0.2813 - accuracy: 0.8689 - val_loss: 0.3306 - val_accuracy: 0.8520
Epoch 7/50
438/438 [==============================] - 522s 1s/step - loss: 0.2688 - accuracy